<a href="https://colab.research.google.com/github/YoruAlptraum/imersao-ia-2025/blob/main/consulta_rapida_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sobre o projeto

Projeto de chatbot com Google Gemini desenvolvido para a Imersão IA da Alura.

O objetivo principal deste bot para Telegram é auxiliar o usuário a tomar uma decisão mais informada sobre a necessidade de buscar atendimento médico. Através de uma conversa simples e interativa no Telegram, o bot fará perguntas relevantes sobre seus sintomas e sua condição atual. Ao final da interação, o usuário receberá uma orientação inicial sobre se é recomendado procurar um médico o mais breve possível ou se a situação pode ser monitorada antes de tomar uma decisão.

**Importante:** Este bot oferece uma triagem inicial e **não substitui, de forma alguma, uma consulta médica profissional**. A avaliação fornecida aqui é apenas uma orientação preliminar e não deve ser considerada um diagnóstico ou recomendação de tratamento. Em caso de dúvidas persistentes ou sintomas graves, procure sempre um médico.

## Setup e imports

In [172]:
!pip -q install google-genai
!pip -q install telebot
!pip -q install google-adk

In [173]:
import os
import telebot # bot do telegram
from google import genai
from google.colab import userdata

# Framework ADK de agentes do Google
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
warnings.filterwarnings("ignore")

# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
# Configura o cliente da SDK do Gemini
MODEL_ID = "gemini-2.0-flash"
client = genai.Client()

## Funções

In [106]:
session_service = InMemorySessionService()
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, user_id: str, message_text: str, session_id: str = "default") -> str:
    # Tenta obter a sessão existente ou cria uma nova se não existir

    session = session_service.get_session(app_name=agent.name, user_id=user_id, session_id=session_id)
    if session is None:
        session = session_service.create_session(app_name=agent.name, user_id=user_id, session_id=session_id)

    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id=user_id, session_id=session_id, new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [18]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [168]:
##########################################
# --- Agente 1: Pesquisador --- #
##########################################
def agente_pesquisador(msg, user_id):
    agent = Agent(
        name="agente_pesquisador",
        model="gemini-2.0-flash",
        instruction="""
        Seu papel é identificar sintomas e fazer uma pesquisa sobre o problema do usuário para repassar a informação para outro agente que irá conversar com o usuário
        use o google search para ajudar a reduzir a lista de possiveis causas para o problema do usuário.

        retorne uma lista das até 5 mais provaveis causas que estejam relacionadas aos sintomas informados, com nivel de urgência, e uma ou duas perguntas que podem ajudar a melhor identificar o problema.

        Faça perguntas relevantes mas não faça muitas perguntas de uma vez, limite para no máximo 3 perguntas por vez e mantenha um tom de conversa.

        mantenha a informação concisa e objetiva.
        """,
        description="Agente que faz a pesquisa sobre o problema do usuário",
        tools=[google_search]
    )
    entrada = f"Mensagem inicial: {msg}"
    # Executa o agente
    pesquisa = call_agent(agent, user_id, entrada)
    return pesquisa

In [169]:
##########################################
# --- Agente 2: Mediador --- #
##########################################
def agente_mediador(msg, pesquisa, user_id):
    agent = Agent(
        name="agente_mediador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um chatbot de Telegram chamado Consulta Rápida
        Seu papel é ajudar o usuário a tomar uma decisão mais informada se deve buscar atendimento médico ou se pode esperar um pouco os sintomas serem corrigidos.
        Analise as informações coletadas e utilize elas para formular uma nova pergunta ao usuário.

        Auxilie o usuário a tomar uma decisão quando tiver informações suficientes.

        Se a decisão final for que é recomendado ao usuário ir ao médico pergunte ao usuário se ele deseja buscar médicos/clinicas próximas.

        Mantenha as mensagens sucintas e sem termos tecnicos.
        """,
        description="Agente que simplifica a informação pesquisada e interage com o usuário",
    )
    entrada = f"Informações coletadas: {pesquisa}, Mensagem inicial do usuário {msg}"
    # Executa o agente
    pesquisa = call_agent(agent, user_id, entrada)
    return pesquisa

In [170]:
##########################################
# --- Agente 3: Localizador --- #
##########################################
def agente_localizador(msg, pesquisa, user_id):
    agent = Agent(
        name="agente_localizador",
        model="gemini-2.0-flash",
        instruction="""
        Seu papel é buscar médicos/clinicas relevantes ao problema do usuário próximas a localização dele.

        Caso o usuário responda que quer buscar locais próximos peça a ele que informe a localização para buscar clínicas relevantes pŕoximas e
        use o google_search para encontrar endereços, telefones e horários de funcionamento de locais relevantes e retorne os 5 mais próximos e bem avaliados do google em formato de lista.
        Priorize resultados que incluem informações de contato e localização.

        Formatação da lista:

        1.Clinica:
            - Endereço:
            - Telefone:
            - Horário de funcionamento:
            - Sobre:

        Se o usuário ainda não tiver respondido a pergunta sobre buscar locais próximos apenas repita o "Retorno inicial"
        """,
        description="Agente que busca médicos/clinicas próximas",
        tools=[google_search]
    )
    entrada = f"Retorno inicial: {pesquisa}, Mensagem do usuário {msg}"
    # Executa o agente
    pesquisa = call_agent(agent, user_id, entrada)
    return pesquisa

## Implementação para rodar com telegram

In [ ]:
bot = telebot.TeleBot(userdata.get('TELEGRAM_BOT_KEY'))
data_de_hoje = date.today().strftime("%d/%m/%Y")
session_service = InMemorySessionService()

# mensagens específicas usando '/' . ex: /start e /help
@bot.message_handler(['start', 'help'])
def start(msg:telebot.types.Message):
    bot.reply_to(msg, """
    Olá, sou um assistente virtual para te ajudar a decidir quando procurar um médico.
    Converse comigo e responda algumas perguntas sobre seus sintomas. Ao final, receba uma orientação inicial sobre a necessidade de atendimento médico.

    Atenção: Lembre-se que sou apenas uma triagem inicial e não substituo uma consulta médica profissional. Em caso de dúvidas ou sintomas graves, procure sempre um médico.
    """)

# reage a qualquer outra mensagem
@bot.message_handler()
def handler(msg:telebot.types.Message):
    user_id = str(msg.from_user.id)
    pesquisa = agente_pesquisador(msg.text, user_id)
    response = agente_mediador(msg, pesquisa, user_id)
    localiza = agente_localizador(msg, response, user_id)
    bot.reply_to(msg, localiza)

bot.infinity_polling()